In [1]:
from utils.config import Config
from utils.data import Data
import pandas as pd
import numpy as np

In [2]:
c = Config()

In [3]:
desc_ie = pd.read_csv(c.descriptors_IE)
desc_c33 = pd.read_csv(c.descriptors_C33)

In [4]:
df_300K = Data().get300K()

set index
time to load 3.32


 # plan
 1. check the unique monolayer ids in descriptors and compare with the 300K and 18M, how much is missing?
 2. some comparison is needed between the two descriptors files, do we take everything?
 3. now becuase i have the IE and C33, should be able to do some feature important analysis.
 4. sample the 300K file, is there a test for samlping
 5. fit a UMAP, fit different UMAPs
 6. visualise

 # EDA

## Descriptors 

### check counts and column names 

In [5]:
print(f'descriptors in the IE file {desc_ie.shape}')
print(f'descriptors in the C33 file {desc_c33.shape}')
print(f'number of unique monlayers used in 300K: {df_300K.monolayer1.unique().size}')

descriptors in the IE file (771, 44)
descriptors in the C33 file (771, 98)
number of unique monlayers used in 300K: 770


In [6]:
desc_ie.columns

Index(['Monolayer', 'Atom_HardnessRB_StDiv',
       'Atom_Inv_Diff_ENARrev_ENGordyRev_Range',
       'Atom_Inv_Diff_ENGordyRev_EA_StDiv',
       'Atom_Inv_Diff_ENGordyRev_IP5_StDiv',
       'Atom_Inv_Diff_ENGordyRev_IP6_StDiv', 'Atom_Inv_Diff_IP7_IP8_StDiv',
       'Atom_Inv_Mult_EA_ThermalExpansion_Range',
       'Atom_Inv_Mult_IP2_LatticeConstant_Min',
       'Atom_Inv_Mult_SpeedSound_Zeff_Median',
       'Atom_Log_Mult_ENGordyRev_IP6_Range', 'Atom_Log_Mult_IP7_Period_Range',
       'Atom_Log_Mult_SpeedSound_RadiiCordero08_Median',
       'Atom_Log_Ratio_IP2_IP3_StDiv', 'Atom_Log_Ratio_IP6_IP7_Max',
       'Atom_Log_Ratio_PoissonRatio_Ntotal_Range', 'Atom_Mult_EA_dHAtom_Max',
       'Atom_Mult_EA_dHAtom_StDiv', 'Atom_Mult_EA_dHFus_StDiv',
       'Atom_Mult_IP8_SpeedSound_Min', 'Atom_Mult_IP8_SpeedSound_StDiv',
       'Atom_Mult_LatticeConstant_Density_StDiv',
       'Atom_Mult_LatticeConstant_RadiiBatsanov_Min',
       'Atom_Mult_ThermalCond_YoungsModule_StDiv',
       'Atom_Ratio_De

In [7]:
desc_ie.rename(columns={'Monolayer':'monolayer'}, inplace=True)
desc_c33.rename(columns={'Monolayer': 'monolayer'}, inplace=True)


Get the monolayers out of the 300K file to be used later to check against the descriptors files, we want to know if all monolayers in the 300K have corresponding descriptors 

In [8]:
monolayers_in_300K = pd.Series(df_300K.monolayer1.unique(), name='monolayer')
print(monolayers_in_300K)

0            Hf3Te2
1           W1S2-T1
2          Ti2Ge2O6
3      Ca2La2I10-T1
4         Ca2La2I10
           ...     
765       Hg2I2N2O6
766       Ta2Ni4Te4
767      Eu2Al4Cl16
768       Ba1Sb2F12
769          Au2Se2
Name: monolayer, Length: 770, dtype: object


### The comparison dataframe 

The comparison dataframe joins the monolayer names form the 300K with the ones in the both descriptor files and determ 

In [9]:
comparison_df = pd.DataFrame(
    monolayers_in_300K) \
    .rename(columns={'monolayer':'key'}) \
    .merge(
        desc_ie.monolayer,
        suffixes=['_300K','_desc_ie'], 
        how='outer',
        left_on=['key'], 
        right_on=['monolayer'],
        indicator='merge_300k_descie').rename(columns={'monolayer':'desc_ie_monolayer'}) \
    .merge(
        desc_c33.monolayer, 
        suffixes=['_300K','_desc_c33'],
        how='outer',
        left_on=['key'],
        right_on=['monolayer'],
        indicator='merge_300k_descc33').rename(columns={'monolayer':'desc_c33_monolayer'})


In [10]:
(comparison_df).head()

,key,desc_ie_monolayer,merge_300k_descie,desc_c33_monolayer,merge_300k_descc33
0,Hf3Te2,Hf3Te2,both,Hf3Te2,both
1,W1S2-T1,W1S2-T1,both,W1S2-T1,both
2,Ti2Ge2O6,Ti2Ge2O6,both,Ti2Ge2O6,both
3,Ca2La2I10-T1,Ca2La2I10-T1,both,Ca2La2I10-T1,both
4,Ca2La2I10,Ca2La2I10,both,Ca2La2I10,both


In [11]:
(comparison_df).tail()

,key,desc_ie_monolayer,merge_300k_descie,desc_c33_monolayer,merge_300k_descc33
767,Eu2Al4Cl16,Eu2Al4Cl16,both,Eu2Al4Cl16,both
768,Ba1Sb2F12,Ba1Sb2F12,both,Ba1Sb2F12,both
769,Au2Se2,Au2Se2,both,Au2Se2,both
770,NaN,Co2Sb4Br4O6,right_only,NaN,left_only
771,NaN,NaN,NaN,Co2Sb4Br4O6,right_only


### Compare monoalayer names from the three sets
**key == True** -> monolayer key is in the 300K file, 

**False == False** -> monolayer key is not in the 300K file.

we want to see that all monolayer keys exist in both the 300K file and the descriptors files to know that we have the descriptors for the monolayers used in 300K. 

we dont want to see monolayers in 300K (key == True) that only appear in the left_only set, which means that we have it in the 300K file but no corresponding descriptors in the descriptors file 

In [12]:
comparison_df \
    .groupby([(comparison_df.key.isna() == False), comparison_df.merge_300k_descie]) \
    ['key','merge_300k_descie'] \
    .count() \
    .replace(np.nan, 0)

key  merge_300k_descie
key   merge_300k_descie                          
False left_only            0.0                0.0
      right_only           0.0                1.0
      both                 0.0                0.0
True  left_only            0.0                0.0
      right_only           0.0                0.0
      both               770.0              770.0

In [13]:
# key is na means that monolayer is not in the 300K but was found in one of the desc files 
comparison_df \
    .groupby([comparison_df.key.isna() == False, comparison_df.merge_300k_descc33]) \
    ['key','merge_300k_descc33'] \
    .count() \
    .replace(np.nan, 0)


key  merge_300k_descc33
key   merge_300k_descc33                           
False left_only             0.0                 1.0
      right_only            0.0                 1.0
      both                  0.0                 0.0
True  left_only             0.0                 0.0
      right_only            0.0                 0.0
      both                770.0               770.0

> test confirms that all monolayers in the 300K have corrsponding descriptors

## Columns of Descriptors 

here we want to look at the columns in the two descriptor files, page 3 of the paper states that the **42 descriptors** are used for the interlayer energy predictions, and **89 descriptors** for C33

In [14]:
print(f'{desc_ie.columns.size} columns in IE descriptors file') 
print(f'{desc_c33.columns.size} columns in C33 descriptors file')

44 columns in IE descriptors file
98 columns in C33 descriptors file


> I need to create a series of column names that combines both so we can use later for merging with 300K to bring the correct columns without duplication 

must come back to this, things to check: 
1. compare column names in both files 
    * create a new dataframe with descriptor name as index/column 0
    * indicator column for whether this descriptor is in both files, left_only or right_only
1. compare values of descriptors
    * for each descriptor in common
    * extract an ordered series of descriptor values, order by monolayer name
    * compare the two series
    * add the result of the comparison to the descriptor names dataset 

In [15]:
pd.Series(desc_ie.columns, index=desc_ie.columns).align(
    pd.Series(desc_c33.columns, index=desc_c33.columns))

(Atom_EA_Min                                                                  NaN
 Atom_HardnessRB_StDiv                                      Atom_HardnessRB_StDiv
 Atom_IP8_Min                                                                 NaN
 Atom_Inv_Diff_ENARrev_ENGordyRev_Range    Atom_Inv_Diff_ENARrev_ENGordyRev_Range
 Atom_Inv_Diff_ENGordyRev_EA_StDiv              Atom_Inv_Diff_ENGordyRev_EA_StDiv
                                                            ...                  
 SG187                                                                      SG187
 Unnamed: 43                                                          Unnamed: 43
 Unnamed: 97                                                                  NaN
 c                                                                            NaN
 monolayer                                                              monolayer
 Length: 125, dtype: object,
 Atom_EA_Min                                         Atom_EA_Min
 Ato

In [16]:
# returns index that is in ie_columns but not in c33_columns
desc_ie.columns.difference(desc_c33.columns)
# if we take the c33_columns, and add to it the result of this difference operation, we get a list of all unique values across the two

Index(['Atom_Inv_Diff_ENARrev_ENGordyRev_Range',
       'Atom_Inv_Diff_ENGordyRev_EA_StDiv',
       'Atom_Inv_Diff_ENGordyRev_IP5_StDiv',
       'Atom_Inv_Diff_ENGordyRev_IP6_StDiv',
       'Atom_Inv_Mult_IP2_LatticeConstant_Min',
       'Atom_Inv_Mult_SpeedSound_Zeff_Median',
       'Atom_Log_Mult_ENGordyRev_IP6_Range', 'Atom_Log_Mult_IP7_Period_Range',
       'Atom_Log_Mult_SpeedSound_RadiiCordero08_Median',
       'Atom_Log_Ratio_IP2_IP3_StDiv', 'Atom_Log_Ratio_IP6_IP7_Max',
       'Atom_Mult_EA_dHAtom_Max', 'Atom_Mult_EA_dHAtom_StDiv',
       'Atom_Mult_IP8_SpeedSound_Min', 'Atom_Mult_IP8_SpeedSound_StDiv',
       'Atom_Mult_LatticeConstant_Density_StDiv',
       'Atom_Mult_ThermalCond_YoungsModule_StDiv',
       'Atom_Ratio_Density_TDebye0_Min', 'Atom_Ratio_ENARrev_YoungsModule_Min',
       'Atom_Ratio_dHFus_SpeedSound_StDiv', 'EState_Topo_Ni_Zeff',
       'EState_Topo_Pd_Zeff', 'EState_Topo_Te_dHAtom', 'Moment_SpeedSound_1',
       'Moment_TCurie_1', 'SG187', 'Unnamed: 43'],
    

In [35]:
difference_columns = pd.Series(desc_ie.columns.difference(desc_c33.columns))
c33_columns = pd.Series(desc_c33.columns)

print(c33_columns)
print(difference_columns)

all_columns = c33_columns.append(difference_columns, ignore_index=True)
type(all_columns)

0                       monolayer
1                     Atom_EA_Min
2           Atom_HardnessRB_StDiv
3                    Atom_IP8_Min
4     Atom_Inv_Diff_IP2_IP4_StDiv
                 ...             
93                       Ratio_ac
94                       Ratio_bc
95                          SG164
96                              c
97                    Unnamed: 97
Length: 98, dtype: object
0             Atom_Inv_Diff_ENARrev_ENGordyRev_Range
1                  Atom_Inv_Diff_ENGordyRev_EA_StDiv
2                 Atom_Inv_Diff_ENGordyRev_IP5_StDiv
3                 Atom_Inv_Diff_ENGordyRev_IP6_StDiv
4              Atom_Inv_Mult_IP2_LatticeConstant_Min
5               Atom_Inv_Mult_SpeedSound_Zeff_Median
6                 Atom_Log_Mult_ENGordyRev_IP6_Range
7                     Atom_Log_Mult_IP7_Period_Range
8     Atom_Log_Mult_SpeedSound_RadiiCordero08_Median
9                       Atom_Log_Ratio_IP2_IP3_StDiv
10                        Atom_Log_Ratio_IP6_IP7_Max
11               

/home/mutaz/.local/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


pandas.core.series.Series

In [67]:
len(difference_columns)

27

In [61]:
desc_ie[difference_columns].head()

,Atom_Inv_Diff_ENARrev_ENGordyRev_Range,Atom_Inv_Diff_ENGordyRev_EA_StDiv,Atom_Inv_Diff_ENGordyRev_IP5_StDiv,Atom_Inv_Diff_ENGordyRev_IP6_StDiv,Atom_Inv_Mult_IP2_LatticeConstant_Min,Atom_Inv_Mult_SpeedSound_Zeff_Median,Atom_Log_Mult_ENGordyRev_IP6_Range,Atom_Log_Mult_IP7_Period_Range,Atom_Log_Mult_SpeedSound_RadiiCordero08_Median,Atom_Log_Ratio_IP2_IP3_StDiv,...,Atom_Ratio_Density_TDebye0_Min,Atom_Ratio_ENARrev_YoungsModule_Min,Atom_Ratio_dHFus_SpeedSound_StDiv,EState_Topo_Ni_Zeff,EState_Topo_Pd_Zeff,EState_Topo_Te_dHAtom,Moment_SpeedSound_1,Moment_TCurie_1,SG187,Unnamed: 43
0,0.03942,0.11466,0.22918,0.19705,0.23920,0.48319,0.21346,0.11842,0.56026,0.01596,...,0.00840,0.00734,0.06082,0.0,0.0,0.0,-1.97540,0.0,0,NaN
1,0.00261,0.11970,0.21606,0.00941,0.40860,0.06370,0.02923,0.00927,0.81742,0.06878,...,0.01920,0.08081,0.00000,0.0,0.0,0.0,0.88510,0.0,0,NaN
2,0.53339,0.07534,0.13968,0.14296,0.23920,0.16034,0.13091,0.12344,0.62618,0.05172,...,0.00308,0.15596,0.04767,0.0,0.0,0.0,-12.92227,0.0,0,NaN
3,0.00215,0.06046,0.11091,0.00985,0.27327,0.03533,0.01857,0.00197,0.91968,0.07456,...,0.07585,0.72586,0.00000,0.0,0.0,0.0,-0.93949,0.0,0,NaN
4,0.53353,0.10706,0.10141,0.14117,0.19345,0.11512,0.16919,0.06623,0.82268,0.03545,...,0.19525,0.06068,0.03165,0.0,0.0,0.0,1.04053,0.0,0,NaN


In [62]:
desc_c33[difference_columns].head() # returns error as difference columns are not in c33

KeyError: "None of [Index(['Atom_Inv_Diff_ENARrev_ENGordyRev_Range',\n       'Atom_Inv_Diff_ENGordyRev_EA_StDiv',\n       'Atom_Inv_Diff_ENGordyRev_IP5_StDiv',\n       'Atom_Inv_Diff_ENGordyRev_IP6_StDiv',\n       'Atom_Inv_Mult_IP2_LatticeConstant_Min',\n       'Atom_Inv_Mult_SpeedSound_Zeff_Median',\n       'Atom_Log_Mult_ENGordyRev_IP6_Range', 'Atom_Log_Mult_IP7_Period_Range',\n       'Atom_Log_Mult_SpeedSound_RadiiCordero08_Median',\n       'Atom_Log_Ratio_IP2_IP3_StDiv', 'Atom_Log_Ratio_IP6_IP7_Max',\n       'Atom_Mult_EA_dHAtom_Max', 'Atom_Mult_EA_dHAtom_StDiv',\n       'Atom_Mult_IP8_SpeedSound_Min', 'Atom_Mult_IP8_SpeedSound_StDiv',\n       'Atom_Mult_LatticeConstant_Density_StDiv',\n       'Atom_Mult_ThermalCond_YoungsModule_StDiv',\n       'Atom_Ratio_Density_TDebye0_Min', 'Atom_Ratio_ENARrev_YoungsModule_Min',\n       'Atom_Ratio_dHFus_SpeedSound_StDiv', 'EState_Topo_Ni_Zeff',\n       'EState_Topo_Pd_Zeff', 'EState_Topo_Te_dHAtom', 'Moment_SpeedSound_1',\n       'Moment_TCur

In [63]:
desc_c33.index

RangeIndex(start=0, stop=771, step=1)

In [64]:
desc_ie.index

RangeIndex(start=0, stop=771, step=1)

In [50]:
# cleanup the all columns to remove
all_columns.str.contains('monolayer').sum()
all_columns.str.contains('Unnamed').sum()
all_columns.str.match('c').sum()
filter_unwanted_column_names = all_columns.str.contains('monolayer') | \
    all_columns.str.contains('Unnamed') | \
    all_columns.str.match('c')
print(f'number of columns to drop: {filter_unwanted_column_names.sum()}')
#print(all_columns)
all_columns = all_columns[filter_unwanted_column_names == False]
all_columns.to_csv('./descriptors/all_descriptors300K_columns.csv')

number of columns to drop: 4


/home/mutaz/.local/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [55]:
# load_column names 
column_names = pd.read_csv('./descriptors/all_descriptors300K_columns.csv',header=None, squeeze=True, index_col=0) 
print(column_names)

0
1                      Atom_EA_Min
2            Atom_HardnessRB_StDiv
3                     Atom_IP8_Min
4      Atom_Inv_Diff_IP2_IP4_StDiv
5      Atom_Inv_Diff_IP5_IP6_StDiv
                  ...             
119            EState_Topo_Pd_Zeff
120          EState_Topo_Te_dHAtom
121            Moment_SpeedSound_1
122                Moment_TCurie_1
123                          SG187
Name: 1, Length: 121, dtype: object


In [59]:
print(f'type: {type(column_names)}')
print(column_names[:10])

desc_c33[column_names[:10]]

type: <class 'pandas.core.series.Series'>
0
1                                 Atom_EA_Min
2                       Atom_HardnessRB_StDiv
3                                Atom_IP8_Min
4                 Atom_Inv_Diff_IP2_IP4_StDiv
5                 Atom_Inv_Diff_IP5_IP6_StDiv
6                 Atom_Inv_Diff_IP7_IP8_StDiv
7     Atom_Inv_Mult_EA_ThermalExpansion_Range
8         Atom_Inv_Mult_ENGhosh2010_Group_Max
9            Atom_Inv_Mult_IP8_HeatCap_Median
10    Atom_Inv_Mult_IP8_LatticeConstant_StDiv
Name: 1, dtype: object


,Atom_EA_Min,Atom_HardnessRB_StDiv,Atom_IP8_Min,Atom_Inv_Diff_IP2_IP4_StDiv,Atom_Inv_Diff_IP5_IP6_StDiv,Atom_Inv_Diff_IP7_IP8_StDiv,Atom_Inv_Mult_EA_ThermalExpansion_Range,Atom_Inv_Mult_ENGhosh2010_Group_Max,Atom_Inv_Mult_IP8_HeatCap_Median,Atom_Inv_Mult_IP8_LatticeConstant_StDiv
0,0.00508,0.05207,0.10588,0.15975,0.10389,0.31814,0.12941,0.36899,0.21552,0.09350
1,0.03790,0.05361,0.31834,0.02446,0.09623,0.02571,0.00000,0.44173,0.49157,0.03459
2,0.01848,0.06427,0.12580,0.07760,0.20656,0.17580,0.00000,0.94545,0.21552,0.15668
3,0.04085,0.04245,0.18493,0.01384,0.10333,0.00816,0.00000,0.39757,0.37393,0.05650
4,0.01848,0.12388,0.12580,0.13403,0.19625,0.26333,0.06563,0.94545,0.17672,0.22518
5,0.00954,0.07079,0.16574,0.13567,0.09461,0.27288,0.00000,0.37993,0.50818,0.01980
6,0.00076,0.05083,0.15138,0.01118,0.03576,0.10561,0.43488,0.38282,0.35691,0.02832
7,-0.05076,0.08739,0.16543,0.03851,0.01308,0.13310,0.04547,0.34146,0.21552,0.15301
8,0.02386,0.03463,0.11008,0.14302,0.15733,0.22344,0.00000,0.39914,0.29031,0.08786
9,0.00000,0.03210,0.13639,0.08738,0.22241,0.08999,0.00000,0.44958,0.44179,0.05661


## plots

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt 



In [ ]:
sns.pairplot(desc_ie.iloc[:,:20])


In [ ]:
g=sns.pairplot(desc_ie.iloc[:,:10],diag_kind="kde")
plt.show()

# Descriptors master data 

1. melt (unpivot, wide to long) the descriptors 
2. concat the two descriptors 
3. remove duplicates by monolayer 
    * after testing that descritors are equal across files 
    * drop duplicates by monolayer name and descriptor name 
4. pivot to return to original structure
5. save master descriptors to file 

# join descriptors with 300K

need to join on monolayer 1 and monolayer 2
then need to add the values of the descriptor for both , would be nice of there is a way to do this when doing a join. 

1. use pandas melt() to unpivot on monolayer1 and 2 columns (from wide to long) 
1. join with master descriptors to bring in all descriptor values 
1. group and aggregate on uuid
    * sum() for descriptors
    * first and last for layer names 
    * first for IE, C33 and others 
    



 > there is no real need to include the layer names or the IE/C33 predictions in the last step as these can easily be obtained from the original set, so maybe just focus the process on producing the summation of the descriptor values, then append layer names, IE and CC33 values using  join() or concat(axis=1) 

In [231]:
df_300K_small = df_300K.head(10)

In [232]:
df_300K_small.reset_index(level=0, inplace=True)

In [233]:
df_300K_small

,uid,bilayer,monolayer1,monolayer2,IE,IE_error,IE_rel_error,C33,C33_error,C33_rel_err
0,Hf3Te2_Mo1S2-T1,Hf3Te2_Mo1S2-T1,Hf3Te2,Mo1S2-T1,-0.698046,0.081306,0.058239,36.954759,6.369837,0.086184
1,Hf3Te2_W1S2-T1,W1S2-T1_Hf3Te2,W1S2-T1,Hf3Te2,-0.677167,0.080142,0.059174,44.432743,7.523579,0.084663
2,Cr1S2-T1_Hf3Te2,Hf3Te2_Cr1S2-T1,Hf3Te2,Cr1S2-T1,-0.661212,0.074767,0.056538,54.217246,11.076879,0.102153
3,Ca2La2I10-T1_Ti2Ge2O6,Ti2Ge2O6_Ca2La2I10-T1,Ti2Ge2O6,Ca2La2I10-T1,-0.658413,0.117797,0.089456,49.623362,18.454026,0.185941
4,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1,Ta3Te1I7,-0.657006,0.083776,0.063756,28.505406,4.794372,0.084096
5,Ca2La2I10_Ti2Ge2O6,Ca2La2I10_Ti2Ge2O6,Ca2La2I10,Ti2Ge2O6,-0.654798,0.122613,0.093626,107.532152,43.877526,0.204020
6,Ca2La2I10_Ti6H4O14,Ca2La2I10_Ti6H4O14,Ca2La2I10,Ti6H4O14,-0.653412,0.124932,0.095599,142.223580,51.729440,0.181860
7,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1,Ti6H4O14,-0.650855,0.126579,0.097241,75.621323,31.855674,0.210626
8,Ca2La2I10-T1_Te2Au1,Te2Au1_Ca2La2I10-T1,Te2Au1,Ca2La2I10-T1,-0.649248,0.102973,0.079302,30.607404,7.194662,0.117531
9,B1N1-T1_Mo1S2-T1,B1N1-T1_Mo1S2-T1,B1N1-T1,Mo1S2-T1,-0.648136,0.060763,0.046875,10.133768,3.384325,0.166983


In [234]:
df_long = pd.melt(
    df_300K_small, 
    id_vars=['uid'], 
    value_vars=['monolayer1', 'monolayer2'], 
    var_name='layer', 
    value_name='layer_name').sort_values('uid').drop(['layer'],axis=1)

In [235]:
df_long

,uid,layer_name
9,B1N1-T1_Mo1S2-T1,B1N1-T1
19,B1N1-T1_Mo1S2-T1,Mo1S2-T1
4,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1
14,Ca2La2I10-T1_Ta3Te1I7,Ta3Te1I7
8,Ca2La2I10-T1_Te2Au1,Te2Au1
18,Ca2La2I10-T1_Te2Au1,Ca2La2I10-T1
3,Ca2La2I10-T1_Ti2Ge2O6,Ti2Ge2O6
13,Ca2La2I10-T1_Ti2Ge2O6,Ca2La2I10-T1
7,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1
17,Ca2La2I10-T1_Ti6H4O14,Ti6H4O14


In [236]:
df_300K_small

,uid,bilayer,monolayer1,monolayer2,IE,IE_error,IE_rel_error,C33,C33_error,C33_rel_err
0,Hf3Te2_Mo1S2-T1,Hf3Te2_Mo1S2-T1,Hf3Te2,Mo1S2-T1,-0.698046,0.081306,0.058239,36.954759,6.369837,0.086184
1,Hf3Te2_W1S2-T1,W1S2-T1_Hf3Te2,W1S2-T1,Hf3Te2,-0.677167,0.080142,0.059174,44.432743,7.523579,0.084663
2,Cr1S2-T1_Hf3Te2,Hf3Te2_Cr1S2-T1,Hf3Te2,Cr1S2-T1,-0.661212,0.074767,0.056538,54.217246,11.076879,0.102153
3,Ca2La2I10-T1_Ti2Ge2O6,Ti2Ge2O6_Ca2La2I10-T1,Ti2Ge2O6,Ca2La2I10-T1,-0.658413,0.117797,0.089456,49.623362,18.454026,0.185941
4,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1,Ta3Te1I7,-0.657006,0.083776,0.063756,28.505406,4.794372,0.084096
5,Ca2La2I10_Ti2Ge2O6,Ca2La2I10_Ti2Ge2O6,Ca2La2I10,Ti2Ge2O6,-0.654798,0.122613,0.093626,107.532152,43.877526,0.204020
6,Ca2La2I10_Ti6H4O14,Ca2La2I10_Ti6H4O14,Ca2La2I10,Ti6H4O14,-0.653412,0.124932,0.095599,142.223580,51.729440,0.181860
7,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1,Ti6H4O14,-0.650855,0.126579,0.097241,75.621323,31.855674,0.210626
8,Ca2La2I10-T1_Te2Au1,Te2Au1_Ca2La2I10-T1,Te2Au1,Ca2La2I10-T1,-0.649248,0.102973,0.079302,30.607404,7.194662,0.117531
9,B1N1-T1_Mo1S2-T1,B1N1-T1_Mo1S2-T1,B1N1-T1,Mo1S2-T1,-0.648136,0.060763,0.046875,10.133768,3.384325,0.166983


In [237]:
columns_to_bring_back = ['uid','bilayer', 'monolayer1', 'monolayer2', 'IE', 'IE_error','IE_rel_error','C33', 'C33_error', 'C33_rel_err']
df_300K_small[columns_to_bring_back]

,uid,bilayer,monolayer1,monolayer2,IE,IE_error,IE_rel_error,C33,C33_error,C33_rel_err
0,Hf3Te2_Mo1S2-T1,Hf3Te2_Mo1S2-T1,Hf3Te2,Mo1S2-T1,-0.698046,0.081306,0.058239,36.954759,6.369837,0.086184
1,Hf3Te2_W1S2-T1,W1S2-T1_Hf3Te2,W1S2-T1,Hf3Te2,-0.677167,0.080142,0.059174,44.432743,7.523579,0.084663
2,Cr1S2-T1_Hf3Te2,Hf3Te2_Cr1S2-T1,Hf3Te2,Cr1S2-T1,-0.661212,0.074767,0.056538,54.217246,11.076879,0.102153
3,Ca2La2I10-T1_Ti2Ge2O6,Ti2Ge2O6_Ca2La2I10-T1,Ti2Ge2O6,Ca2La2I10-T1,-0.658413,0.117797,0.089456,49.623362,18.454026,0.185941
4,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1,Ta3Te1I7,-0.657006,0.083776,0.063756,28.505406,4.794372,0.084096
5,Ca2La2I10_Ti2Ge2O6,Ca2La2I10_Ti2Ge2O6,Ca2La2I10,Ti2Ge2O6,-0.654798,0.122613,0.093626,107.532152,43.877526,0.204020
6,Ca2La2I10_Ti6H4O14,Ca2La2I10_Ti6H4O14,Ca2La2I10,Ti6H4O14,-0.653412,0.124932,0.095599,142.223580,51.729440,0.181860
7,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1,Ti6H4O14,-0.650855,0.126579,0.097241,75.621323,31.855674,0.210626
8,Ca2La2I10-T1_Te2Au1,Te2Au1_Ca2La2I10-T1,Te2Au1,Ca2La2I10-T1,-0.649248,0.102973,0.079302,30.607404,7.194662,0.117531
9,B1N1-T1_Mo1S2-T1,B1N1-T1_Mo1S2-T1,B1N1-T1,Mo1S2-T1,-0.648136,0.060763,0.046875,10.133768,3.384325,0.166983


In [238]:
# when using the join method, the other dataframe has to have an index to join on
df_long.join(df_300K_small[['uid','IE']].set_index('uid'), on='uid')

,uid,layer_name,IE
9,B1N1-T1_Mo1S2-T1,B1N1-T1,-0.648136
19,B1N1-T1_Mo1S2-T1,Mo1S2-T1,-0.648136
4,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1,-0.657006
14,Ca2La2I10-T1_Ta3Te1I7,Ta3Te1I7,-0.657006
8,Ca2La2I10-T1_Te2Au1,Te2Au1,-0.649248
18,Ca2La2I10-T1_Te2Au1,Ca2La2I10-T1,-0.649248
3,Ca2La2I10-T1_Ti2Ge2O6,Ti2Ge2O6,-0.658413
13,Ca2La2I10-T1_Ti2Ge2O6,Ca2La2I10-T1,-0.658413
7,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1,-0.650855
17,Ca2La2I10-T1_Ti6H4O14,Ti6H4O14,-0.650855


In [239]:
df_long.rename(columns={'layer_name':'monolayer'}) \
    .join(desc_ie[['monolayer','Atom_HardnessRB_StDiv']].set_index('monolayer'), on='monolayer') 

,uid,monolayer,Atom_HardnessRB_StDiv
9,B1N1-T1_Mo1S2-T1,B1N1-T1,0.12441
19,B1N1-T1_Mo1S2-T1,Mo1S2-T1,0.00000
4,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1,0.02959
14,Ca2La2I10-T1_Ta3Te1I7,Ta3Te1I7,0.05353
8,Ca2La2I10-T1_Te2Au1,Te2Au1,0.00084
18,Ca2La2I10-T1_Te2Au1,Ca2La2I10-T1,0.02959
3,Ca2La2I10-T1_Ti2Ge2O6,Ti2Ge2O6,0.10941
13,Ca2La2I10-T1_Ti2Ge2O6,Ca2La2I10-T1,0.02959
7,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1,0.02959
17,Ca2La2I10-T1_Ti6H4O14,Ti6H4O14,0.11528


In [240]:
ie_columns=['monolayer', 'Atom_Inv_Diff_ENARrev_ENGordyRev_Range',
       'Atom_Inv_Diff_ENGordyRev_EA_StDiv',
       'Atom_Inv_Diff_ENGordyRev_IP5_StDiv',
       'Atom_Inv_Diff_ENGordyRev_IP6_StDiv',
       'Atom_Inv_Mult_IP2_LatticeConstant_Min',
       'Atom_Inv_Mult_SpeedSound_Zeff_Median']

master_df = df_long.rename(columns={'layer_name':'monolayer'}) \
    .join(desc_ie[ie_columns].set_index('monolayer'), on='monolayer') \
    .groupby('uid', as_index=False).sum()

,uid,Atom_Inv_Diff_ENARrev_ENGordyRev_Range,Atom_Inv_Diff_ENGordyRev_EA_StDiv,Atom_Inv_Diff_ENGordyRev_IP5_StDiv,Atom_Inv_Diff_ENGordyRev_IP6_StDiv,Atom_Inv_Mult_IP2_LatticeConstant_Min,Atom_Inv_Mult_SpeedSound_Zeff_Median
0,B1N1-T1_Mo1S2-T1,0.00846,0.19566,0.25513,0.08433,0.40759,0.22145
1,Ca2La2I10-T1_Ta3Te1I7,0.00865,0.23293,0.20242,0.18708,0.66371,0.17457
2,Ca2La2I10-T1_Te2Au1,0.00879,0.17420,0.15518,0.11667,0.66079,0.12644
3,Ca2La2I10-T1_Ti2Ge2O6,0.01450,0.23214,0.17796,0.13254,0.65159,0.51970
4,Ca2La2I10-T1_Ti6H4O14,0.02072,0.22463,0.16020,0.12462,0.65159,0.51970
5,Ca2La2I10_Ti2Ge2O6,0.01450,0.23214,0.17796,0.13254,0.65159,0.51970
6,Ca2La2I10_Ti6H4O14,0.02072,0.22463,0.16020,0.12462,0.65159,0.51970
7,Cr1S2-T1_Hf3Te2,0.01190,0.23198,0.05200,0.15235,0.47727,0.16827
8,Hf3Te2_Mo1S2-T1,0.00823,0.20469,0.13983,0.21749,0.50216,0.17024
9,Hf3Te2_W1S2-T1,0.00395,0.21119,0.19191,0.26896,0.49940,0.25501


In [242]:
df_long.rename(columns={'layer_name':'monolayer'}) \
    .join(desc_ie[ie_columns].set_index('monolayer'), on='monolayer') \
    .groupby('uid', as_index=False).sum() \
    .join(df_300K_small[columns_to_bring_back].set_index('uid'), on='uid')

,uid,Atom_Inv_Diff_ENARrev_ENGordyRev_Range,Atom_Inv_Diff_ENGordyRev_EA_StDiv,Atom_Inv_Diff_ENGordyRev_IP5_StDiv,Atom_Inv_Diff_ENGordyRev_IP6_StDiv,Atom_Inv_Mult_IP2_LatticeConstant_Min,Atom_Inv_Mult_SpeedSound_Zeff_Median,bilayer,monolayer1,monolayer2,IE,IE_error,IE_rel_error,C33,C33_error,C33_rel_err
0,B1N1-T1_Mo1S2-T1,0.00846,0.19566,0.25513,0.08433,0.40759,0.22145,B1N1-T1_Mo1S2-T1,B1N1-T1,Mo1S2-T1,-0.648136,0.060763,0.046875,10.133768,3.384325,0.166983
1,Ca2La2I10-T1_Ta3Te1I7,0.00865,0.23293,0.20242,0.18708,0.66371,0.17457,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1,Ta3Te1I7,-0.657006,0.083776,0.063756,28.505406,4.794372,0.084096
2,Ca2La2I10-T1_Te2Au1,0.00879,0.17420,0.15518,0.11667,0.66079,0.12644,Te2Au1_Ca2La2I10-T1,Te2Au1,Ca2La2I10-T1,-0.649248,0.102973,0.079302,30.607404,7.194662,0.117531
3,Ca2La2I10-T1_Ti2Ge2O6,0.01450,0.23214,0.17796,0.13254,0.65159,0.51970,Ti2Ge2O6_Ca2La2I10-T1,Ti2Ge2O6,Ca2La2I10-T1,-0.658413,0.117797,0.089456,49.623362,18.454026,0.185941
4,Ca2La2I10-T1_Ti6H4O14,0.02072,0.22463,0.16020,0.12462,0.65159,0.51970,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1,Ti6H4O14,-0.650855,0.126579,0.097241,75.621323,31.855674,0.210626
5,Ca2La2I10_Ti2Ge2O6,0.01450,0.23214,0.17796,0.13254,0.65159,0.51970,Ca2La2I10_Ti2Ge2O6,Ca2La2I10,Ti2Ge2O6,-0.654798,0.122613,0.093626,107.532152,43.877526,0.204020
6,Ca2La2I10_Ti6H4O14,0.02072,0.22463,0.16020,0.12462,0.65159,0.51970,Ca2La2I10_Ti6H4O14,Ca2La2I10,Ti6H4O14,-0.653412,0.124932,0.095599,142.223580,51.729440,0.181860
7,Cr1S2-T1_Hf3Te2,0.01190,0.23198,0.05200,0.15235,0.47727,0.16827,Hf3Te2_Cr1S2-T1,Hf3Te2,Cr1S2-T1,-0.661212,0.074767,0.056538,54.217246,11.076879,0.102153
8,Hf3Te2_Mo1S2-T1,0.00823,0.20469,0.13983,0.21749,0.50216,0.17024,Hf3Te2_Mo1S2-T1,Hf3Te2,Mo1S2-T1,-0.698046,0.081306,0.058239,36.954759,6.369837,0.086184
9,Hf3Te2_W1S2-T1,0.00395,0.21119,0.19191,0.26896,0.49940,0.25501,W1S2-T1_Hf3Te2,W1S2-T1,Hf3Te2,-0.677167,0.080142,0.059174,44.432743,7.523579,0.084663


In [243]:
df_300K_small[columns_to_bring_back]

,uid,bilayer,monolayer1,monolayer2,IE,IE_error,IE_rel_error,C33,C33_error,C33_rel_err
0,Hf3Te2_Mo1S2-T1,Hf3Te2_Mo1S2-T1,Hf3Te2,Mo1S2-T1,-0.698046,0.081306,0.058239,36.954759,6.369837,0.086184
1,Hf3Te2_W1S2-T1,W1S2-T1_Hf3Te2,W1S2-T1,Hf3Te2,-0.677167,0.080142,0.059174,44.432743,7.523579,0.084663
2,Cr1S2-T1_Hf3Te2,Hf3Te2_Cr1S2-T1,Hf3Te2,Cr1S2-T1,-0.661212,0.074767,0.056538,54.217246,11.076879,0.102153
3,Ca2La2I10-T1_Ti2Ge2O6,Ti2Ge2O6_Ca2La2I10-T1,Ti2Ge2O6,Ca2La2I10-T1,-0.658413,0.117797,0.089456,49.623362,18.454026,0.185941
4,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1_Ta3Te1I7,Ca2La2I10-T1,Ta3Te1I7,-0.657006,0.083776,0.063756,28.505406,4.794372,0.084096
5,Ca2La2I10_Ti2Ge2O6,Ca2La2I10_Ti2Ge2O6,Ca2La2I10,Ti2Ge2O6,-0.654798,0.122613,0.093626,107.532152,43.877526,0.204020
6,Ca2La2I10_Ti6H4O14,Ca2La2I10_Ti6H4O14,Ca2La2I10,Ti6H4O14,-0.653412,0.124932,0.095599,142.223580,51.729440,0.181860
7,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1_Ti6H4O14,Ca2La2I10-T1,Ti6H4O14,-0.650855,0.126579,0.097241,75.621323,31.855674,0.210626
8,Ca2La2I10-T1_Te2Au1,Te2Au1_Ca2La2I10-T1,Te2Au1,Ca2La2I10-T1,-0.649248,0.102973,0.079302,30.607404,7.194662,0.117531
9,B1N1-T1_Mo1S2-T1,B1N1-T1_Mo1S2-T1,B1N1-T1,Mo1S2-T1,-0.648136,0.060763,0.046875,10.133768,3.384325,0.166983


In [ ]:
# join master_df with to reconsutrct the final complete data set of descriptor features and output labels 